In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
dataset_size = 40000

ids = list(range(0, 70000))
random.shuffle(ids)

all_id = ids[:dataset_size]

valid_split = 0.1
test_split = 0.1

v_index = int(len(all_id)*(1-valid_split-test_split))
t_index = int(len(all_id)*(1-test_split))

train_ids = all_id[:v_index]
valid_ids = all_id[v_index:t_index]
test_ids = all_id[t_index:]

print("Length of train set: " + str(len(train_ids)))
print("Length of validation set: " + str(len(valid_ids)))
print("Length of test set: " + str(len(test_ids)))

partition = {'train': train_ids, 'validation': valid_ids, 'test': test_ids}

Length of train set: 32000
Length of validation set: 4000
Length of test set: 4000


In [3]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=10, dim=(256,256,1), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *(256,256,1)), dtype=float)
        Y = np.empty((self.batch_size, *(256,256,2)), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            img = img_to_array(load_img('C:/Users/lakas/Documents/deeplearning/imcol-master/dataset/' + str(ID) + '.jpg'))
            img = 1.0/255*img
            img = rgb2lab(img)
            
            gray = img[:,:,0]
            ab = img[:,:,1:] / 128
            
            # Store sample
            X[i,] = gray.reshape((256,256,1))

            # Store class
            Y[i,] = ab

        return X, Y

In [4]:
patience=30
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

In [5]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

class Swish(Activation):
    
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

def swish(x):
    # Your activation function specialties here
    A = (K.sigmoid(x) * x)
    return A

get_custom_objects().update({'swish': Swish(swish)})

In [6]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [7]:
encoder_input = Input(shape=(256, 256, 1,), name='encoder_input')
encoder_output = Conv2D(64, (3,3), activation='swish', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='swish', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='swish', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='swish', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='swish', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='swish', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='swish', padding='same')(encoder_output)


encoder_output = Conv2D(256, (3,3), activation='swish', padding='same')(encoder_output)

#Decoder
decoder_output = Conv2D(128, (3,3), activation='swish', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='swish', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='swish', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='swish', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=encoder_input, outputs=decoder_output)

In [8]:
# Image transformer
#datagen = ImageDataGenerator(
#        shear_range=0.2,
#        zoom_range=0.2,
#        rotation_range=20,
#        horizontal_flip=True)

# Generate training data
#batch_size = 10
#def image_a_b_gen(batch_size):
#    for batch in datagen.flow(Xtrain, batch_size=batch_size):
#        lab_batch = rgb2lab(batch)
#        X_batch = lab_batch[:,:,:,0]
#        Y_batch = lab_batch[:,:,:,1:] / 128
#        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)


training_generator = DataGenerator(partition['train'], batch_size=50)
valid_generator = DataGenerator(partition['validation'], batch_size=50)
# Train model      
tensorboard = TensorBoard(log_dir="/output/beta_run")
model.compile(optimizer='adam', loss='mean_absolute_error')
model.fit_generator(training_generator, validation_data=valid_generator,
                    epochs=100, steps_per_epoch=len(train_ids)//50, validation_steps=len(valid_ids)//50,
                   callbacks=[checkpointer, early_stopping])

Epoch 1/100
640/640 [==============================] - 758s 1s/step - loss: 0.1023 - val_loss: 0.0962

Epoch 00001: val_loss improved from inf to 0.09621, saving model to weights.hdf5
Epoch 2/100
640/640 [==============================] - 761s 1s/step - loss: 0.0959 - val_loss: 0.0954

Epoch 00002: val_loss improved from 0.09621 to 0.09544, saving model to weights.hdf5
Epoch 3/100
640/640 [==============================] - 762s 1s/step - loss: 0.0957 - val_loss: 0.0954

Epoch 00003: val_loss improved from 0.09544 to 0.09536, saving model to weights.hdf5
Epoch 4/100
640/640 [==============================] - 756s 1s/step - loss: 0.0952 - val_loss: 0.0949

Epoch 00004: val_loss improved from 0.09536 to 0.09490, saving model to weights.hdf5
Epoch 5/100
640/640 [==============================] - 778s 1s/step - loss: 0.0946 - val_loss: 0.0947

Epoch 00005: val_loss improved from 0.09490 to 0.09470, saving model to weights.hdf5
Epoch 6/100
640/640 [==============================] - 731s 1s/s

640/640 [==============================] - 716s 1s/step - loss: 0.0506 - val_loss: 0.0967

Epoch 00050: val_loss did not improve from 0.08881
Epoch 51/100
640/640 [==============================] - 717s 1s/step - loss: 0.0502 - val_loss: 0.0956

Epoch 00051: val_loss did not improve from 0.08881
Epoch 52/100
640/640 [==============================] - 714s 1s/step - loss: 0.0496 - val_loss: 0.0949

Epoch 00052: val_loss did not improve from 0.08881
Epoch 53/100
640/640 [==============================] - 721s 1s/step - loss: 0.0492 - val_loss: 0.0957

Epoch 00053: val_loss did not improve from 0.08881
Epoch 54/100
640/640 [==============================] - 716s 1s/step - loss: 0.0489 - val_loss: 0.0962

Epoch 00054: val_loss did not improve from 0.08881
Epoch 00054: early stopping


In [9]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [10]:
# Test images
#Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
#Xtest = Xtest.reshape(Xtest.shape+(1,))
#Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
#Ytest = Ytest / 128
#print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

In [11]:
# Change to '/data/images/Test/' to use all the 500 images
model = load_model('weights.hdf5')

color_me = []
for filename in os.listdir('Test/'):
	color_me.append(img_to_array(load_img('Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
	cur = np.zeros((256, 256, 3))
	cur[:,:,0] = color_me[i][:,:,0]
	cur[:,:,1:] = output[i]
	imsave("result/img_"+str(i)+".png", lab2rgb(cur))

c:\users\lakas\appdata\local\programs\python\python36\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 512)       1180160   
__________

In [13]:
def get_model_memory_usage(batch_size, model):
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

get_model_memory_usage(100, model)

5.772